## This file contains model training on data already cleaned and transformed and merged from two datasets Aplication and Amplitude

The full transformation process is available in run_inference.py file

In [138]:
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, recall_score
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from optuna.visualization import plot_pareto_front
from imblearn.over_sampling import SMOTE



In [27]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/final_dataset_v2.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
df.head()

,Unnamed: 0.1,APPLICATIONID,CREATE_DATETIME_x,TOTALAMOUNT,SUM_CREDIT_KZT,DM5DPD1GCVPSUM,DM5DPD1SALARYSUM,DM5EXPSUM,DM5INCSUM,DM5INC,...,n_unique_event_type_3d,n_unique_ip_3d,n_event_id_3d,country_kz_ratio_30d,n_unique_device_id_30d,n_unique_event_type_30d,n_unique_ip_30d,n_event_id_30d,last_os_name_30d,most_common_os_name_30d
0,0,Д\286\011639474,17.10.2024 12:16,150000,150000,457066.8,0.0,2874498.7,2861073.5,0,...,51.0,2.0,75.0,1.0,1.0,52.0,5.0,105.0,ios,ios
1,1,Д\286\011668478,21.10.2024 8:31,300000,300000,68850.0,0.0,558734.6,564659.8,0,...,62.0,1.0,90.0,1.0,1.0,62.0,1.0,90.0,ios,ios
2,2,Д\286\011681466,22.10.2024 10:05,600000,368858,NaN,0.0,1044469.5,1023278.0,0,...,55.0,2.0,117.0,1.0,1.0,58.0,7.0,145.0,android,android
3,3,Д\286\011721797,26.10.2024 14:13,200000,217391,NaN,0.0,1048790.4,1046686.8,0,...,49.0,3.0,56.0,1.0,1.0,53.0,9.0,282.0,android,android
4,4,Д\286\012045593,01.12.2024 19:14,150000,150000,NaN,0.0,982182.2,942380.7,0,...,1.0,1.0,2.0,1.0,2.0,2.0,2.0,3.0,android,android


In [29]:
df.isna().sum()

,0
Unnamed: 0.1,0
APPLICATIONID,0
CREATE_DATETIME_x,0
TOTALAMOUNT,0
SUM_CREDIT_KZT,0
...,...
n_unique_event_type_30d,9583
n_unique_ip_30d,9583
n_event_id_30d,9583
last_os_name_30d,9583


In [31]:
df.columns

Index(['Unnamed: 0.1', 'APPLICATIONID', 'CREATE_DATETIME_x', 'TOTALAMOUNT',
       'SUM_CREDIT_KZT', 'DM5DPD1GCVPSUM', 'DM5DPD1SALARYSUM', 'DM5EXPSUM',
       'DM5INCSUM', 'DM5INC', 'DM5EXP', 'DM7INC', 'DM7EXP', 'DM6SCOREN6PD',
       'DM6SCOREN6', 'FINALKDN', 'CREDITTERM_RBL0', 'CLI_AGE',
       'DEPENDANT_COUNT', 'IS_SERVWS_MNLI', 'target_x',
       'PURPOSE_LOAN_На потребительские цели',
       'PURPOSE_LOAN_На рефинансирование займа',
       'PURPOSE_LOAN_На рефинансирование займа и потребительские цели',
       'OPV_REASON_NEW САЙТ', 'OPV_REASON_Онлайн СБОЛ', 'KANAL_PRODAZH_B-Bank',
       'KANAL_PRODAZH_QR', 'KANAL_PRODAZH_WEB',
       'APPLICATION_ISA0AUTO_Зарплатники', 'APPLICATION_ISA0AUTO_Улица',
       'GENDER_Женский', 'GENDER_Мужской', 'BKI_Отсутствует', 'BKI_Плохая',
       'BKI_Средняя', 'BKI_Хорошая', 'VKI_Отсутствует', 'VKI_Плохая',
       'VKI_Средняя', 'VKI_Хорошая', 'MARITALSTATUS_Женат/Замужем',
       'MARITALSTATUS_Разведен/Разведена', 'MARITALSTATUS_Холост/Не за

In [32]:
feature_columns = ['TOTALAMOUNT',
       'SUM_CREDIT_KZT', 'DM5DPD1GCVPSUM', 'DM5DPD1SALARYSUM', 'DM5EXPSUM',
       'DM5INCSUM', 'DM5INC', 'DM5EXP', 'DM7INC', 'DM7EXP', 'DM6SCOREN6PD',
       'DM6SCOREN6', 'FINALKDN', 'CREDITTERM_RBL0', 'CLI_AGE',
       'DEPENDANT_COUNT', 'IS_SERVWS_MNLI',
       'PURPOSE_LOAN_На потребительские цели',
       'PURPOSE_LOAN_На рефинансирование займа',
       'PURPOSE_LOAN_На рефинансирование займа и потребительские цели',
       'OPV_REASON_NEW САЙТ', 'OPV_REASON_Онлайн СБОЛ', 'KANAL_PRODAZH_B-Bank',
       'KANAL_PRODAZH_QR', 'KANAL_PRODAZH_WEB',
       'APPLICATION_ISA0AUTO_Зарплатники', 'APPLICATION_ISA0AUTO_Улица',
       'GENDER_Женский', 'GENDER_Мужской', 'BKI_Отсутствует', 'BKI_Плохая',
       'BKI_Средняя', 'BKI_Хорошая', 'VKI_Отсутствует', 'VKI_Плохая',
       'VKI_Средняя', 'VKI_Хорошая', 'MARITALSTATUS_Женат/Замужем',
       'MARITALSTATUS_Разведен/Разведена', 'MARITALSTATUS_Холост/Не замужем',
       'country_kz_ratio_1d', 'n_unique_device_id_1d',
       'n_unique_event_type_1d', 'n_unique_ip_1d', 'n_event_id_1d',
       'country_kz_ratio_2d', 'n_unique_device_id_2d',
       'n_unique_event_type_2d', 'n_unique_ip_2d', 'n_event_id_2d',
       'country_kz_ratio_3d', 'n_unique_device_id_3d',
       'n_unique_event_type_3d', 'n_unique_ip_3d', 'n_event_id_3d',
       'country_kz_ratio_30d', 'n_unique_device_id_30d',
       'n_unique_event_type_30d', 'n_unique_ip_30d', 'n_event_id_30d']

In [33]:
df.columns

Index(['Unnamed: 0.1', 'APPLICATIONID', 'CREATE_DATETIME_x', 'TOTALAMOUNT',
       'SUM_CREDIT_KZT', 'DM5DPD1GCVPSUM', 'DM5DPD1SALARYSUM', 'DM5EXPSUM',
       'DM5INCSUM', 'DM5INC', 'DM5EXP', 'DM7INC', 'DM7EXP', 'DM6SCOREN6PD',
       'DM6SCOREN6', 'FINALKDN', 'CREDITTERM_RBL0', 'CLI_AGE',
       'DEPENDANT_COUNT', 'IS_SERVWS_MNLI', 'target_x',
       'PURPOSE_LOAN_На потребительские цели',
       'PURPOSE_LOAN_На рефинансирование займа',
       'PURPOSE_LOAN_На рефинансирование займа и потребительские цели',
       'OPV_REASON_NEW САЙТ', 'OPV_REASON_Онлайн СБОЛ', 'KANAL_PRODAZH_B-Bank',
       'KANAL_PRODAZH_QR', 'KANAL_PRODAZH_WEB',
       'APPLICATION_ISA0AUTO_Зарплатники', 'APPLICATION_ISA0AUTO_Улица',
       'GENDER_Женский', 'GENDER_Мужской', 'BKI_Отсутствует', 'BKI_Плохая',
       'BKI_Средняя', 'BKI_Хорошая', 'VKI_Отсутствует', 'VKI_Плохая',
       'VKI_Средняя', 'VKI_Хорошая', 'MARITALSTATUS_Женат/Замужем',
       'MARITALSTATUS_Разведен/Разведена', 'MARITALSTATUS_Холост/Не за

In [34]:
df['new_os'] = df.apply(lambda x: 1 if x['most_common_os_name_30d'] == x['last_os_name_30d'] else 0, axis=1)

In [35]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['last_os_name_30d_encoded'] = le.fit_transform(df['last_os_name_30d'])
df['most_common_os_name_30d_encoded'] = le.fit_transform(df['most_common_os_name_30d'])

In [127]:
df_train = df[df['data_x'] == 'train']
df_test = df[df['data_x'] == 'test']
df_val = df[df['data_x'] == 'valid']

In [126]:
df['data_x'].value_counts()

,count
data_x,
train,18386
valid,6379
test,5528


In [58]:
X_train = df_train[feature_columns]
y_train = df_train['target_x']

In [59]:
X_test = df_test[feature_columns]
y_test = df_test['target_x']

In [60]:
X_val_pred = df_val[feature_columns]
y_val_pred = df_val['target_x']

In [61]:
y_train.value_counts()

,count
target_x,
0.0,18046
1.0,340


In [63]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)


In [64]:
y_train.value_counts()

,count
target_x,
0.0,11548
1.0,218


In [65]:
y_val.value_counts()

,count
target_x,
0.0,2888
1.0,54


In [66]:
X_train = X_train.fillna(-1)
X_val = X_val.fillna(-1)
X_val_pred = X_val_pred.fillna(-1)
X_test = X_test.fillna(-1)

In [67]:
smote = SMOTE(sampling_strategy=0.05, random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print("Shape of X_train after SMOTE:", X_train_res.shape)
print("Shape of y_train after SMOTE:", y_train_res.shape)

print("\nValue counts of y_train after SMOTE:")
print(y_train_res.value_counts())

Shape of X_train after SMOTE: (12125, 60)
Shape of y_train after SMOTE: (12125,)

Value counts of y_train after SMOTE:
target_x
0.0    11548
1.0      577
Name: count, dtype: int64


In [68]:
# !pip install optuna
# !pip install lightgbm

In [70]:
def objective(trial, X, y):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "is_unbalance": True,  # Handle class imbalance
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []

    for train_idx, valid_idx in skf.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        model = LGBMClassifier(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            eval_metric="auc",
            callbacks=[
                early_stopping(stopping_rounds=100),
                log_evaluation(period=0)
            ]
        )

        y_pred = model.predict(X_valid)
        auc = roc_auc_score(y_valid, y_pred)
        aucs.append(auc)

    return sum(aucs) / len(aucs)

In [71]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X_train_res, y_train_res)
study.optimize(func, n_trials=50)

[I 2025-06-13 19:21:35,404] A new study created in memory with name: LGBM Classifier


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.973179
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[89]	valid_0's auc: 0.982356
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[90]	valid_0's auc: 0.975468
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.95419
Training until validation scores don't improve for 100 rounds


[I 2025-06-13 19:21:37,734] Trial 0 finished with value: 0.8982278522508332 and parameters: {'learning_rate': 0.19747787739160147, 'num_leaves': 122, 'max_depth': 10, 'feature_fraction': 0.6972579277097649, 'bagging_fraction': 0.6043612024966911, 'bagging_freq': 1, 'min_data_in_leaf': 167, 'lambda_l1': 2.344094927446552, 'lambda_l2': 4.00268438701331}. Best is trial 0 with value: 0.8982278522508332.


Did not meet early stopping. Best iteration is:
[99]	valid_0's auc: 0.983382
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[99]	valid_0's auc: 0.973706
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[76]	valid_0's auc: 0.986561
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[90]	valid_0's auc: 0.974704
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[66]	valid_0's auc: 0.961911
Training until validation scores don't improve for 100 rounds


[I 2025-06-13 19:21:40,840] Trial 1 finished with value: 0.9133101913071856 and parameters: {'learning_rate': 0.12009855084398728, 'num_leaves': 171, 'max_depth': 8, 'feature_fraction': 0.8358182841523452, 'bagging_fraction': 0.9461098512712965, 'bagging_freq': 3, 'min_data_in_leaf': 117, 'lambda_l1': 1.3311216865235642, 'lambda_l2': 1.8591204691019592}. Best is trial 0 with value: 0.8982278522508332.


...

Did not meet early stopping. Best iteration is:
[90]	valid_0's auc: 0.987149
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[88]	valid_0's auc: 0.971493
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[57]	valid_0's auc: 0.986911
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[72]	valid_0's auc: 0.970235
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.966533
Training until validation scores don't improve for 100 rounds


[I 2025-06-13 19:23:51,207] Trial 48 finished with value: 0.8945993187255006 and parameters: {'learning_rate': 0.190781091819576, 'num_leaves': 121, 'max_depth': 12, 'feature_fraction': 0.6914277822119077, 'bagging_fraction': 0.7351520744136989, 'bagging_freq': 6, 'min_data_in_leaf': 37, 'lambda_l1': 3.1339078501055098, 'lambda_l2': 4.224707314686816}. Best is trial 24 with value: 0.8919640649463169.


Did not meet early stopping. Best iteration is:
[98]	valid_0's auc: 0.983834
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.975788
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[95]	valid_0's auc: 0.984118
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.973503
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[70]	valid_0's auc: 0.961466
Training until validation scores don't improve for 100 rounds


[I 2025-06-13 19:23:54,486] Trial 49 finished with value: 0.8950566961011684 and parameters: {'learning_rate': 0.17358398275792264, 'num_leaves': 171, 'max_depth': 12, 'feature_fraction': 0.6951772738751604, 'bagging_fraction': 0.8623292810624325, 'bagging_freq': 6, 'min_data_in_leaf': 24, 'lambda_l1': 3.140901899219507, 'lambda_l2': 4.1692952863951245}. Best is trial 24 with value: 0.8919640649463169.


Did not meet early stopping. Best iteration is:
[89]	valid_0's auc: 0.984883


In [72]:
print(f"\tBest value (auc): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")


	Best value (auc): 0.89196
	Best params:
		learning_rate: 0.19349739366176544
		num_leaves: 62
		max_depth: 11
		feature_fraction: 0.7961256231877186
		bagging_fraction: 0.7129690764883406
		bagging_freq: 9
		min_data_in_leaf: 25
		lambda_l1: 4.03526964074153
		lambda_l2: 4.399397958647765


In [73]:
optuna.visualization.plot_optimization_history(study)

In [74]:
optuna.visualization.plot_param_importances(study)

In [75]:
best_params = study.best_params

In [76]:
final_model = LGBMClassifier(**best_params)

final_model.fit(
    X_train_res, y_train_res,
    eval_set=[(X_val, y_val)],
    eval_metric="auc",
    callbacks=[
        early_stopping(50),
        log_evaluation(50)
    ]
)


Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.957686	valid_0's binary_logloss: 0.046021
Early stopping, best iteration is:
[28]	valid_0's auc: 0.961572	valid_0's binary_logloss: 0.048128


LGBMClassifier(bagging_fraction=0.7129690764883406, bagging_freq=9,
               feature_fraction=0.7961256231877186, lambda_l1=4.03526964074153,
               lambda_l2=4.399397958647765, learning_rate=0.19349739366176544,
               max_depth=11, min_data_in_leaf=25, num_leaves=62)

In [78]:
def print_metrics(model, X, y, label):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]  # For AUC
    auc = roc_auc_score(y, y_proba)
    acc = accuracy_score(y, y_pred)
    print(f"{label} AUC: {auc:.4f} | Accuracy: {acc:.4f}")

print_metrics(final_model, X_train, y_train, "Train")
print_metrics(final_model, X_val, y_val, "Valid")
print_metrics(final_model, X_test, y_test, "Test")


Train AUC: 0.9717 | Accuracy: 0.9873
Valid AUC: 0.9616 | Accuracy: 0.9861
Test AUC: 0.8625 | Accuracy: 0.9904


In [79]:
importance = final_model.feature_importances_
feature_names = final_model.feature_name_

feat_imp_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importance
}).sort_values(by="importance", ascending=False)

print(feat_imp_df)


                                              feature  importance
1                                      SUM_CREDIT_KZT         101
44                                      n_event_id_1d          67
14                                            CLI_AGE          64
12                                           FINALKDN          53
5                                           DM5INCSUM          49
10                                       DM6SCOREN6PD          47
4                                           DM5EXPSUM          44
2                                      DM5DPD1GCVPSUM          44
0                                         TOTALAMOUNT          43
57                            n_unique_event_type_30d          34
13                                    CREDITTERM_RBL0          33
58                                    n_unique_ip_30d          27
15                                    DEPENDANT_COUNT          23
42                             n_unique_event_type_1d          22
11        

In [80]:
feat_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importance
})

important_features = feat_df[feat_df["importance"] > 7]["feature"].tolist()

In [87]:
important_features =[
 'TOTALAMOUNT',
 'DM5DPD1GCVPSUM',
 'DM5EXPSUM',
 'DM5INCSUM',
 'DM5EXP',
 'DM7EXP',
 'DM6SCOREN6PD',
 'DM6SCOREN6',
 'FINALKDN',
 'CREDITTERM_RBL0',
 'CLI_AGE',
 'DEPENDANT_COUNT',
 'PURPOSE_LOAN_На потребительские цели',
 'KANAL_PRODAZH_QR',
 'GENDER_Мужской',
 'BKI_Плохая',
 'VKI_Хорошая',
 'MARITALSTATUS_Холост/Не замужем',
 'country_kz_ratio_1d',
 'n_unique_device_id_1d',
 'n_unique_event_type_1d',
 'n_unique_ip_1d',
 'n_event_id_1d',
 'n_unique_event_type_2d',
 'n_event_id_2d',
 'n_event_id_3d',
 'country_kz_ratio_30d',
 'n_unique_device_id_30d',
 'n_unique_event_type_30d',
 'n_unique_ip_30d',
 'n_event_id_30d']

In [88]:
df[important_features]

,TOTALAMOUNT,DM5DPD1GCVPSUM,DM5EXPSUM,DM5INCSUM,DM5EXP,DM7EXP,DM6SCOREN6PD,DM6SCOREN6,FINALKDN,CREDITTERM_RBL0,...,n_unique_ip_1d,n_event_id_1d,n_unique_event_type_2d,n_event_id_2d,n_event_id_3d,country_kz_ratio_30d,n_unique_device_id_30d,n_unique_event_type_30d,n_unique_ip_30d,n_event_id_30d
0,150000,457066.8,2874498.7,2861073.5,0,1,0.042531,765.975104,0.4519,12,...,2.0,75.0,51.0,75.0,75.0,1.0,1.0,52.0,5.0,105.0
1,300000,68850.0,558734.6,564659.8,0,0,0.027982,777.614138,0.4247,12,...,1.0,90.0,62.0,90.0,90.0,1.0,1.0,62.0,1.0,90.0
2,600000,NaN,1044469.5,1023278.0,1,0,0.075949,739.240506,0.5000,12,...,2.0,117.0,55.0,117.0,117.0,1.0,1.0,58.0,7.0,145.0
3,200000,NaN,1048790.4,1046686.8,1,0,0.077830,737.735849,0.3266,12,...,3.0,56.0,49.0,56.0,56.0,1.0,1.0,53.0,9.0,282.0
4,150000,NaN,982182.2,942380.7,1,0,0.075949,739.240506,0.3364,12,...,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30288,150000,311175.0,1484270.7,1481724.7,0,1,0.007680,793.855607,NaN,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30289,1500000,NaN,553521.7,553468.8,1,0,0.049884,760.092807,0.3715,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30290,150000,NaN,410992.0,411387.7,0,0,0.027982,777.614138,NaN,24,...,3.0,108.0,46.0,116.0,129.0,1.0,1.0,47.0,7.0,188.0
30291,3708542,88088.9,511921.2,512088.8,0,0,0.147200,682.240000,NaN,60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X_train_res[important_features], y_train_res)
study.optimize(func, n_trials=50)

[I 2025-06-13 19:43:00,822] A new study created in memory with name: LGBM Classifier


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.975825
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[78]	valid_0's auc: 0.980862
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[81]	valid_0's auc: 0.972181
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[82]	valid_0's auc: 0.957135
Training until validation scores don't improve for 100 rounds


[I 2025-06-13 19:43:03,746] Trial 0 finished with value: 0.9196447284195669 and parameters: {'learning_rate': 0.15795550021574914, 'num_leaves': 213, 'max_depth': 3, 'feature_fraction': 0.6404588722313999, 'bagging_fraction': 0.803477865293546, 'bagging_freq': 10, 'min_data_in_leaf': 30, 'lambda_l1': 2.870209246229, 'lambda_l2': 1.1751363407632436}. Best is trial 0 with value: 0.9196447284195669.


Did not meet early stopping. Best iteration is:
[83]	valid_0's auc: 0.980171
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.969881
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[88]	valid_0's auc: 0.982718
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[68]	valid_0's auc: 0.976398
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[96]	valid_0's auc: 0.961283
Training until validation scores don't improve for 100 rounds


[I 2025-06-13 19:43:05,655] Trial 1 finished with value: 0.8950489460291987 and parameters: {'learning_rate': 0.18275361095430379, 'num_leaves': 85, 'max_depth': 7, 'feature_fraction': 0.7211043559752601, 'bagging_fraction': 0.737114109199758, 'bagging_freq': 8, 'min_data_in_leaf': 98, 'lambda_l1': 3.897475621915008, 'lambda_l2': 0.22685126473448558}. Best is trial 1 with value: 0.8950489460291987.


...

Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.981407
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[98]	valid_0's auc: 0.971432
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.985658
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.971745
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.95208
Training until validation scores don't improve for 100 rounds


[I 2025-06-13 19:44:57,469] Trial 48 finished with value: 0.9087861330649801 and parameters: {'learning_rate': 0.047229411004203845, 'num_leaves': 131, 'max_depth': 9, 'feature_fraction': 0.6886362085419556, 'bagging_fraction': 0.6835952993982171, 'bagging_freq': 9, 'min_data_in_leaf': 25, 'lambda_l1': 2.4093464860408376, 'lambda_l2': 4.15461584116448}. Best is trial 20 with value: 0.7971777077241811.


Did not meet early stopping. Best iteration is:
[71]	valid_0's auc: 0.982464
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[65]	valid_0's auc: 0.966012
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.979684
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.962503
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[96]	valid_0's auc: 0.95388
Training until validation scores don't improve for 100 rounds


[I 2025-06-13 19:44:59,567] Trial 49 finished with value: 0.9053827078322161 and parameters: {'learning_rate': 0.031674815719050044, 'num_leaves': 156, 'max_depth': 11, 'feature_fraction': 0.7782080524011877, 'bagging_fraction': 0.6227839869768581, 'bagging_freq': 8, 'min_data_in_leaf': 85, 'lambda_l1': 4.087660516529062, 'lambda_l2': 1.2434434334870121}. Best is trial 20 with value: 0.7971777077241811.


Did not meet early stopping. Best iteration is:
[99]	valid_0's auc: 0.978058


In [90]:
best_params = study.best_params

In [92]:
final_model = LGBMClassifier(**best_params)

final_model.fit(
    X_train_res[important_features], y_train_res,
    eval_set=[(X_val[important_features], y_val)],
    eval_metric="auc",
    callbacks=[
        early_stopping(50),
        log_evaluation(50)
    ]
)


Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.942203	valid_0's binary_logloss: 0.0783538
[100]	valid_0's auc: 0.945041	valid_0's binary_logloss: 0.0675164
Did not meet early stopping. Best iteration is:
[81]	valid_0's auc: 0.945547	valid_0's binary_logloss: 0.0709166


LGBMClassifier(bagging_fraction=0.6030035366684824, bagging_freq=9,
               feature_fraction=0.8333050286816874, lambda_l1=4.40969471787694,
               lambda_l2=1.862166602572901, learning_rate=0.010512231844894476,
               max_depth=10, min_data_in_leaf=14, num_leaves=110)

In [93]:
print_metrics(final_model, X_train[important_features], y_train, "Train")
print_metrics(final_model, X_val[important_features], y_val, "Valid")
print_metrics(final_model, X_test[important_features], y_test, "Test")


Train AUC: 0.9440 | Accuracy: 0.9818
Valid AUC: 0.9455 | Accuracy: 0.9813
Test AUC: 0.8547 | Accuracy: 0.9931


In [95]:
len(important_features)

31

In [ ]:
def print_metrics_with_threshold(model, X, y, label, threshold=0.5):
    y_proba = model.predict_proba(X)[:, 1]
    y_pred_threshold = (y_proba >= threshold).astype(int)

    auc = roc_auc_score(y, y_proba)
    acc = accuracy_score(y, y_pred_threshold)
    f1 = f1_score(y, y_pred_threshold)
    recall = recall_score(y, y_pred_threshold)

    print(f"{label} (Threshold={threshold:.2f}) AUC: {auc:.4f} | Accuracy: {acc:.4f} | F1 Score: {f1:.4f} | Recall: {recall:.4f}")


In [100]:
print("\nMetrics with default threshold (0.5):")
print_metrics_with_threshold(final_model, X_train[important_features], y_train, "Train", threshold=0.5)
print_metrics_with_threshold(final_model, X_val[important_features], y_val, "Valid", threshold=0.5)
print_metrics_with_threshold(final_model, X_test[important_features], y_test, "Test", threshold=0.5)

print("\nMetrics with threshold 0.8:")
print_metrics_with_threshold(final_model, X_train[important_features], y_train, "Train", threshold=0.8)
print_metrics_with_threshold(final_model, X_val[important_features], y_val, "Valid", threshold=0.8)
print_metrics_with_threshold(final_model, X_test[important_features], y_test, "Test", threshold=0.8)



Metrics with default threshold (0.5):
Train (Threshold=0.50) AUC: 0.9440 | Accuracy: 0.9818 | F1 Score: 0.0531 | Recall: 0.0275
Valid (Threshold=0.50) AUC: 0.9455 | Accuracy: 0.9813 | F1 Score: 0.0000 | Recall: 0.0000
Test (Threshold=0.50) AUC: 0.8547 | Accuracy: 0.9931 | F1 Score: 0.0000 | Recall: 0.0000

Metrics with threshold 0.8:
Train (Threshold=0.80) AUC: 0.9440 | Accuracy: 0.9815 | F1 Score: 0.0000 | Recall: 0.0000
Valid (Threshold=0.80) AUC: 0.9455 | Accuracy: 0.9816 | F1 Score: 0.0000 | Recall: 0.0000
Test (Threshold=0.80) AUC: 0.8547 | Accuracy: 0.9931 | F1 Score: 0.0000 | Recall: 0.0000


In [102]:
def objective(trial, X_train, y_train, X_val, y_val):
    params = {
        "objective": "binary",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "is_unbalance": True,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 5.0),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 5.0),
    }



    model = LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="auc",
        callbacks=[early_stopping(50), log_evaluation(0)]
    )

    y_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_proba > 0.5).astype(int)

    auc = roc_auc_score(y_val, y_proba)
    f1 = f1_score(y_val, y_pred)

    return auc, f1


In [104]:
study = optuna.create_study(
    directions=["maximize", "maximize"],  # AUC and F1
)
study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=100)


[I 2025-06-13 20:06:09,717] A new study created in memory with name: no-name-509b0040-6766-4577-b794-0d2495dca6b2


Training until validation scores don't improve for 50 rounds


[I 2025-06-13 20:06:10,155] Trial 0 finished with values: [0.954473171232174, 0.2830188679245283] and parameters: {'learning_rate': 0.1678817171853569, 'num_leaves': 125, 'max_depth': 3, 'feature_fraction': 0.9924133727183609, 'bagging_fraction': 0.6754802144057013, 'bagging_freq': 6, 'min_data_in_leaf': 64, 'lambda_l1': 4.655909793101647, 'lambda_l2': 4.8995139812884165}.


Did not meet early stopping. Best iteration is:
[75]	valid_0's auc: 0.954473	valid_0's binary_logloss: 0.180157
Training until validation scores don't improve for 50 rounds


[I 2025-06-13 20:06:10,384] Trial 1 finished with values: [0.8486745921822099, 0.0] and parameters: {'learning_rate': 0.08021601344113144, 'num_leaves': 93, 'max_depth': 4, 'feature_fraction': 0.7993541801847952, 'bagging_fraction': 0.6018967762008544, 'bagging_freq': 7, 'min_data_in_leaf': 148, 'lambda_l1': 3.428610673300085, 'lambda_l2': 2.2665439575893886}.


...

Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.938148	valid_0's binary_logloss: 0.0642972
Training until validation scores don't improve for 50 rounds


[I 2025-06-13 20:06:56,875] Trial 98 finished with values: [0.957724171539961, 0.47560975609756095] and parameters: {'learning_rate': 0.128021809088676, 'num_leaves': 99, 'max_depth': 10, 'feature_fraction': 0.8954651498946518, 'bagging_fraction': 0.6089240657529246, 'bagging_freq': 10, 'min_data_in_leaf': 69, 'lambda_l1': 4.102497241231006, 'lambda_l2': 4.966237373689465}.


Did not meet early stopping. Best iteration is:
[69]	valid_0's auc: 0.957724	valid_0's binary_logloss: 0.0813319
Training until validation scores don't improve for 50 rounds


[I 2025-06-13 20:06:57,858] Trial 99 finished with values: [0.9517607981943161, 0.47058823529411764] and parameters: {'learning_rate': 0.1556990643891862, 'num_leaves': 119, 'max_depth': 11, 'feature_fraction': 0.8198984165355476, 'bagging_fraction': 0.7105639289826764, 'bagging_freq': 2, 'min_data_in_leaf': 82, 'lambda_l1': 3.8488935500103745, 'lambda_l2': 0.21410978624226362}.


Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.951761	valid_0's binary_logloss: 0.0673636


In [105]:
fig = plot_pareto_front(study, target_names=["AUC", "F1"])
fig.show()


In [111]:
best_params = study.trials[51].params
best_params

{'learning_rate': 0.1580027631549685,
 'num_leaves': 198,
 'max_depth': 10,
 'feature_fraction': 0.8839875734474862,
 'bagging_fraction': 0.7625828117841102,
 'bagging_freq': 1,
 'min_data_in_leaf': 135,
 'lambda_l1': 1.7208440020384164,
 'lambda_l2': 4.966237373689465}

In [114]:
final_model = LGBMClassifier(**best_params)

final_model.fit(
    X_train_res, y_train_res,
    eval_set=[(X_val, y_val)],
    eval_metric="auc",
    callbacks=[
        early_stopping(50),
        log_evaluation(50)
    ]
)


Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.964021	valid_0's binary_logloss: 0.0452497
Early stopping, best iteration is:
[32]	valid_0's auc: 0.965855	valid_0's binary_logloss: 0.0475425


LGBMClassifier(bagging_fraction=0.7625828117841102, bagging_freq=1,
               feature_fraction=0.8839875734474862,
               lambda_l1=1.7208440020384164, lambda_l2=4.966237373689465,
               learning_rate=0.1580027631549685, max_depth=10,
               min_data_in_leaf=135, num_leaves=198)

In [115]:
importance = final_model.feature_importances_
feature_names = final_model.feature_name_

feat_imp_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importance
}).sort_values(by="importance", ascending=False)

print(feat_imp_df)


                                              feature  importance
12                                           FINALKDN          76
1                                      SUM_CREDIT_KZT          52
44                                      n_event_id_1d          50
14                                            CLI_AGE          46
0                                         TOTALAMOUNT          45
5                                           DM5INCSUM          38
13                                    CREDITTERM_RBL0          36
2                                      DM5DPD1GCVPSUM          35
10                                       DM6SCOREN6PD          34
4                                           DM5EXPSUM          30
57                            n_unique_event_type_30d          21
58                                    n_unique_ip_30d          19
42                             n_unique_event_type_1d          15
55                               country_kz_ratio_30d          15
59        

In [117]:
print("\nMetrics with default threshold (0.5):")
print_metrics_with_threshold(final_model, X_train, y_train, "Train", threshold=0.5)
print_metrics_with_threshold(final_model, X_val, y_val, "Valid", threshold=0.5)
print_metrics_with_threshold(final_model, X_test, y_test, "Test", threshold=0.5)




Metrics with default threshold (0.5):
Train (Threshold=0.50) AUC: 0.9702 | Accuracy: 0.9876 | F1 Score: 0.5655 | Recall: 0.4358
Valid (Threshold=0.50) AUC: 0.9659 | Accuracy: 0.9857 | F1 Score: 0.4878 | Recall: 0.3704
Test (Threshold=0.50) AUC: 0.8569 | Accuracy: 0.9908 | F1 Score: 0.1053 | Recall: 0.0789


In [118]:
feat_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importance
})

important_features = feat_df[feat_df["importance"] >= 3]["feature"].tolist()

In [121]:
important_features = ['TOTALAMOUNT',
 'DM5DPD1GCVPSUM',
 'DM5EXPSUM',
 'DM5INCSUM',
 'DM5INC',
 'DM5EXP',
 'DM7INC',
 'DM7EXP',
 'DM6SCOREN6PD',
 'DM6SCOREN6',
 'FINALKDN',
 'CREDITTERM_RBL0',
 'CLI_AGE',
 'DEPENDANT_COUNT',
 'PURPOSE_LOAN_На потребительские цели',
 'KANAL_PRODAZH_B-Bank',
 'KANAL_PRODAZH_QR',
 'GENDER_Женский',
 'BKI_Плохая',
 'VKI_Хорошая',
 'MARITALSTATUS_Женат/Замужем',
 'country_kz_ratio_1d',
 'n_unique_device_id_1d',
 'n_unique_event_type_1d',
 'n_unique_ip_1d',
 'n_event_id_1d',
 'n_event_id_2d',
 'country_kz_ratio_3d',
 'n_event_id_3d',
 'country_kz_ratio_30d',
 'n_unique_event_type_30d',
 'n_unique_ip_30d',
 'n_event_id_30d']

In [122]:
final_model = LGBMClassifier(**best_params)

final_model.fit(
    X_train_res[important_features], y_train_res,
    eval_set=[(X_val[important_features], y_val)],
    eval_metric="auc",
    callbacks=[
        early_stopping(50),
        log_evaluation(50)
    ]
)


Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.963688	valid_0's binary_logloss: 0.045385
Early stopping, best iteration is:
[27]	valid_0's auc: 0.965547	valid_0's binary_logloss: 0.0495221


LGBMClassifier(bagging_fraction=0.7625828117841102, bagging_freq=1,
               feature_fraction=0.8839875734474862,
               lambda_l1=1.7208440020384164, lambda_l2=4.966237373689465,
               learning_rate=0.1580027631549685, max_depth=10,
               min_data_in_leaf=135, num_leaves=198)

In [124]:
print("\nMetrics with default threshold (0.5):")
print_metrics_with_threshold(final_model, X_train[important_features], y_train, "Train", threshold=0.5)
print_metrics_with_threshold(final_model, X_val[important_features], y_val, "Valid", threshold=0.5)
print_metrics_with_threshold(final_model, X_test[important_features], y_test, "Test", threshold=0.5)




Metrics with default threshold (0.5):
Train (Threshold=0.50) AUC: 0.9674 | Accuracy: 0.9861 | F1 Score: 0.5060 | Recall: 0.3853
Valid (Threshold=0.50) AUC: 0.9655 | Accuracy: 0.9854 | F1 Score: 0.4691 | Recall: 0.3519
Test (Threshold=0.50) AUC: 0.8441 | Accuracy: 0.9906 | F1 Score: 0.0714 | Recall: 0.0526


In [131]:
X_val_proba = df_val[important_features]

In [129]:
df_val

,Unnamed: 0.1,APPLICATIONID,CREATE_DATETIME_x,TOTALAMOUNT,SUM_CREDIT_KZT,DM5DPD1GCVPSUM,DM5DPD1SALARYSUM,DM5EXPSUM,DM5INCSUM,DM5INC,...,country_kz_ratio_30d,n_unique_device_id_30d,n_unique_event_type_30d,n_unique_ip_30d,n_event_id_30d,last_os_name_30d,most_common_os_name_30d,new_os,last_os_name_30d_encoded,most_common_os_name_30d_encoded
23914,23914,Д\286\012220838,20.12.2024 16:28,1800000,1800000,114750.0,0.0,4945035.7,4943066.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3,3
23915,23915,Д\286\012249879,24.12.2024 11:51,560000,608695,86884.7,0.0,1903547.2,1905279.6,1,...,1.0,1.0,46.0,10.0,198.0,ios,ios,1,2,2
23916,23916,Д\286\012273876,26.12.2024 16:56,1500000,1000000,NaN,0.0,5806962.0,5805763.7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3,3
23917,23917,Д\286\012288632,28.12.2024 16:13,200000,200000,68850.0,0.0,915976.7,915784.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3,3
23918,23918,Д\286\011592446,12.10.2024 11:52,200000,212765,233769.8,0.0,446342.8,447020.4,0,...,1.0,1.0,65.0,2.0,130.0,ios,ios,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30288,30288,Д\286\012275441,26.12.2024 20:52,150000,150000,311175.0,0.0,1484270.7,1481724.7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3,3
30289,30289,Д\286\012141252,12.12.2024 12:19,1500000,1000000,NaN,0.0,553521.7,553468.8,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3,3
30290,30290,Д\286\012161697,14.12.2024 11:52,150000,150000,NaN,0.0,410992.0,411387.7,1,...,1.0,1.0,47.0,7.0,188.0,android,android,1,1,1
30291,30291,Д\286\012248249,24.12.2024 10:04,3708542,1911620,88088.9,0.0,511921.2,512088.8,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3,3


In [132]:
y_val_proba = final_model.predict_proba(X_val_proba)[:, 1]

In [134]:
df_val["y_pred_proba"] = y_val_proba

In [136]:
df_val = df_val[[
 'APPLICATIONID',
 'TOTALAMOUNT',
 'DM5DPD1GCVPSUM',
 'DM5EXPSUM',
 'DM5INCSUM',
 'DM5INC',
 'DM5EXP',
 'DM7INC',
 'DM7EXP',
 'DM6SCOREN6PD',
 'DM6SCOREN6',
 'FINALKDN',
 'CREDITTERM_RBL0',
 'CLI_AGE',
 'DEPENDANT_COUNT',
 'PURPOSE_LOAN_На потребительские цели',
 'KANAL_PRODAZH_B-Bank',
 'KANAL_PRODAZH_QR',
 'GENDER_Женский',
 'BKI_Плохая',
 'VKI_Хорошая',
 'MARITALSTATUS_Женат/Замужем',
 'country_kz_ratio_1d',
 'n_unique_device_id_1d',
 'n_unique_event_type_1d',
 'n_unique_ip_1d',
 'n_event_id_1d',
 'n_event_id_2d',
 'country_kz_ratio_3d',
 'n_event_id_3d',
 'country_kz_ratio_30d',
 'n_unique_event_type_30d',
 'n_unique_ip_30d',
 'n_event_id_30d','y_pred_proba']]

In [137]:
df_val.to_csv("val_pred_proba.csv", index=False)

In [139]:
import joblib

joblib.dump(final_model, "model_antifraud.pkl")

['model_antifraud.pkl']